# GPXファイルの読み込みと他のファイルフォーマットへの変換
+ STEP01:GPXファイルのトラックポイントをGeoDataFrameに変換する
+ STEP02:GeoDataFrameをGeoJSON,CSV,ESRI Shapefileへと変換する

## ライブラリのインストール

In [ ]:
%pip install -q gpxpy
%pip install -q folium
%pip install -q mapclassify

## ライブラリのインポート

In [ ]:
import gpxpy,datetime
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString

## GPXファイルを開く
+ GPXにはwaypoint,route,trackのデータが格納されている可能性があるのでそれらの有無をチェックする

In [ ]:
gpx_file = open('track.gpx', 'r')
gpx = gpxpy.parse(gpx_file)

## 含まれているデータの確認(waypoints,routes,tracks)
print("waypoints:",gpx.waypoints)
print("routes:",gpx.routes)
print("tracks:",gpx.tracks)

## Trackデータを開く
+ 最初の10レコードのみを確認

In [ ]:
gpxtp = gpx.tracks[0].segments[0].points
gpxtp[0:10]

## Trackデータをデータフレームに変換する
+ Trackpointの各レコードには latitude, longitude, elevation, time, speed の要素(情報)が含まれている
+ それらをデータフレームのlat, lon, alt, time, speed 列にそれぞれ格納する

In [ ]:
gpxdf = [[i.latitude, i.longitude, i.elevation, i.time, i.speed] for i in gpxtp]
df = pd.DataFrame(gpxdf, columns=['lon', 'lat', 'alt', 'time', 'speed'])
df.head(5)

+ datatime形式の列情報あるとgeodataframeへの変換処理でこけるので文字列に変更しておく

In [ ]:
df['time'] = df['time'].astype(str)
df.head(5)

## GeoDataFrameに変換する

In [ ]:
geometry = [Point(xy) for xy in zip(df.lat, df.lon)]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.head(5)

## 座標系および投影法をEPSGコードで定義する
+ DataFrameからGeoDataFrameに変換しただけだと投影情報はなにもないので定義する

In [ ]:
print("before",gdf.crs)
gdf.crs = "EPSG:4326"
print("after",gdf.crs)

## とりあえずプロットしてみる
+ 1万件を超えるとJupyterLite環境では描画が無理っす..となることがある
+ そんな時はgoogle colaboratoryなどをつかうとよいですよー

In [ ]:
gdf.explore()

## GeoJSONへの書き出し
+ 汎用的な地理情報ファイルフォーマットであるGeoJSONとして出力

In [ ]:
#GeoJSONファイルとして出力
gdf.to_file('export.geojson',driver='GeoJSON', encoding='UTF-8')

## ESRI Shapefile形式への書き出し
+ 列名の上限が半角英数10文字であることに注意
  + 上限を超えている場合はあらかじめ列名を変更してから変換するとよい
+ 複数のファイルが出力されるので事前にフォルダを生成しておく(べつにやらんでもよいけど)

In [ ]:
#カレントディレクトリにshpフォルダを生成
import os

#フォルダ名を定義
dirname = "shp"

if os.path.isdir(dirname):
    print(dirname + "フォルダがすでにあります")
else:
    os.mkdir(dirname)
    print(dirname + "フォルダを作りました")

#ESRI shapefileとして出力
gdf.to_file("./" + dirname + "/export.shp",encoding='UTF-8')

### CSVファイルとして書き出し
+ ジオメトリ列を削除したものをCSVファイルとして出力
  + 元のGeoDataFrameにはgeometry列は残ったまま(削除したわけではない)

In [ ]:
gdf.drop('geometry', axis=1).to_csv("export.csv")